### Closure Applications (Part 2)

#### Example 1

Let's write a small function that can increment a counter for us - we don't have an incrementor in Python (the ++ operator in Java or C++ for example):

In [1]:
def counter(initial_value):
    # initial_value is a local variable here
    
    def inc(increment=1):
        nonlocal initial_value
        # initial_value is a nonlocal (captured) variable here
        initial_value += increment
        return initial_value
    
    return inc

In [2]:
counter1 = counter(0)

In [3]:
print(counter1(0))

0


In [4]:
print(counter1())

1


In [5]:
print(counter1())

2


In [6]:
print(counter1(8))

10


In [7]:
counter2 = counter(1000)

In [8]:
print(counter2(0))

1000


In [9]:
print(counter2(1))

1001


In [10]:
print(counter2())

1002


In [11]:
print(counter2(220))

1222


As you can see, each closure maintains a reference to the **initial_value** variable that was created when the **counter** function was **called** - each time that function was called, a new local variable **initial_value** was created (with a value assigned from the argument), and it became a nonlocal (captured) variable in the inner scope.

#### Example 2

Let's modify this example to now build something that can run, and maintain a count of how many times we have run some function.

In [12]:
def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
    
    return inner

In [13]:
def add(a, b):
    return a + b

In [14]:
counted_add = counter(add)

And the free variables are:

In [15]:
counted_add.__code__.co_freevars

('cnt', 'fn')

We can now call the `counted_add` function:

In [16]:
counted_add(1, 2)

add has been called 1 times


3

In [17]:
counted_add(2, 3)

add has been called 2 times


5

In [18]:
def mult(a, b, c):
    return a * b * c

In [19]:
counted_mult = counter(mult)

In [20]:
counted_mult(1, 2, 3)

mult has been called 1 times


6

In [21]:
counted_mult(2, 3, 4)

mult has been called 2 times


24

#### Example 3

Let's take this one step further, and actually store the function name and the number of calls in a global dictionary instead of just printing it out all the time.

In [22]:
counters = dict()

def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is global
        return fn(*args, **kwargs)
    
    return inner

In [23]:
counted_add = counter(add)
counted_mult = counter(mult)

Note that `counters` is a **global** variable, and therefore **not** a free variable:

In [24]:
counted_add.__code__.co_freevars

('cnt', 'fn')

In [25]:
counted_mult.__code__.co_freevars

('cnt', 'fn')

We can now call out functions:

In [26]:
counted_add(1, 2)

3

In [27]:
counted_add(2, 3)

5

In [28]:
counted_mult(1, 2, 'a')

'aa'

In [29]:
counted_mult(2, 3, 'b')

'bbbbbb'

In [30]:
counted_mult(1, 1, 'abc')

'abc'

In [31]:
print(counters)

{'add': 2, 'mult': 3}


Of course this relies on us creating the **counters** global variable first and making sure we are naming it that way, so instead, we're going to pass it as an argument to the **counter** function:

In [32]:
def counter(fn, counters):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is nonlocal
        return fn(*args, **kwargs)
    
    return inner

In [33]:
func_counters = dict()
counted_add = counter(add, func_counters)
counted_mult = counter(mult, func_counters)

In [34]:
counted_add.__code__.co_freevars

('cnt', 'counters', 'fn')

As you can see, `counters` is now a free variable.

We can now call our functions:

In [35]:
for i in range(5):
    counted_add(i, i)

for i in range(10):
    counted_mult(i, i, i)

In [36]:
print(func_counters)

{'add': 5, 'mult': 10}


Of course, we don't have to assign the "counted" version of our functions a new name - we can simply assign it to the same name!

In [37]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [38]:
fact = counter(fact, func_counters)

In [39]:
fact(0)

1

In [40]:
fact(3)

6

In [41]:
fact(4)

24

In [42]:
print(func_counters)

{'add': 5, 'mult': 10, 'fact': 3}


Notice, how we essentially **added** some functionality to our `fact` function, without modifying what the `fact` function actually returns.

This leads us straight into our next topic: decorators!